In [6]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

df_relevant_variants = pd.read_csv('stock_out_influence.csv')
# Ensure timestamp is in datetime format
df_relevant_variants['timestamp'] = pd.to_datetime(df_relevant_variants['timestamp'])

# Set index to timestamp for time series analysis
df_relevant_variants.set_index('timestamp', inplace=True)

def fit_sarima_model(data, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0)):
    try:
        model = SARIMAX(data, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False)
        return results
    except ValueError as e:
        print(f"Error fitting SARIMA model: {e}")
        return None


sarima_results = []

for (id, store_skuId, store_storeId), group in df_relevant_variants.groupby(['id', 'store_skuId', 'store_storeId']):
    variant_data = group['store_quantity'].resample('D').sum().fillna(0)  # Resample daily and fill missing values
    if len(variant_data) > 1:  # Ensure there's enough data to fit the model
        sarima_result = fit_sarima_model(variant_data)
        if sarima_result:
            sarima_results.append({
                'id': id,
                'store_skuId': store_skuId,
                'store_storeId': store_storeId,
                'sarima_model': sarima_result
            })

# Example of how to forecast using the fitted SARIMA model for one variant in one store
for result in sarima_results:
    forecast = result['sarima_model'].get_forecast(steps=10)  # Forecast for the next 10 days
    print(f"Forecast for ID: {result['id']}, SKU ID: {result['store_skuId']}, Store ID: {result['store_storeId']}")
    print(forecast.predicted_mean)



C:\Users\FelixNeubauer\AppData\Local\Temp\ipykernel_22436\2274169642.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_relevant_variants = pd.read_csv('stock_out_influence.csv')
C:\Users\FelixNeubauer\anaconda3\envs\py310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\FelixNeubauer\anaconda3\envs\py310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\FelixNeubauer\anaconda3\envs\py310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\FelixNeubauer\anacon

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed